# Load libraries and paths.

In [4]:
%pylab inline
import time

import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, Conv1D, Dropout, ReLU, Flatten, Input, BatchNormalization, Reshape, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import sigmoid, tanh
from keras.losses import binary_crossentropy, mse
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

path_t = "C:\\Users\\cossa\\JupyterNotebook\\ULB\\MA1\\Projet\\ProjetMA1\\C-NMC_Leukemia\\training_data\\"
#hem cells = Human Epidermal Melanocytes 
path_t_fold_0_hem = path_t + "fold_0\\hem\\"
path_t_fold_1_hem = path_t + "fold_1\\hem\\"
path_t_fold_2_hem = path_t + "fold_2\\hem\\"
#all cells = Acute lymphoblastic leukemia : 
path_t_fold_0_all = path_t + "fold_0\\all\\"
path_t_fold_1_all = path_t + "fold_1\\all\\"
path_t_fold_2_all = path_t + "fold_2\\all\\"

path_read_validation = "C:\\Users\\cossa\\JupyterNotebook\\ULB\\MA1\\Projet\\ProjetMA1\\C-NMC_Leukemia\\validation_data\\"
path_all_validation = [path_read_validation + "all\\"]
path_hem_validation = [path_read_validation + "hem\\"]


Populating the interactive namespace from numpy and matplotlib


# Define model

In [2]:
def model():
    model= Sequential()
    model.add(Conv2D(16, (3,3), input_shape=(380,380,3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3,3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3,3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3,3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3,3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu')
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer=Adam(beta_1=0.7,beta_2=0.999), loss=binary_crossentropy, metrics=["acc"])
    return model

# Loading function
### Function to load all paths of the training images, to load images, normalize, resize, get max index, ...

In [2]:
import os
def readImFolderList(path_folder):
    x= []
    for n in range(len(path_folder)):
        for file in os.listdir(path_folder[n]):
            x.append(path_folder[n]+file)
    return np.array(x)
def readAllImFrom(list_of_image):
    x= []
    for n in range(list_of_image.shape[0]):
        x.append((imread(list_of_image[n])))
    return np.array(x)
import cv2 as cv
def norm_and_resize_im(x):
    width = int(380)
    height = int(380)
    dim = (380, 380)
    X_t  = readAllImFrom(x) 
    X_n=[]
    for g in range(X_t.shape[0]):
        img = cv.resize(X_t[g,:,:,:], dim, interpolation = cv.INTER_AREA)
        normalizedImg = cv.normalize(img, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
        X_n.append(normalizedImg)
    return np.array(X_n)
def get_max_index(X):
    index = 0
    max = -1
    for x in range(X.shape[0]):
        if (X[x] > max):
            index = x
            max = X[x]
    return index

# Loading training paths

In [4]:
path_hem =  [path_t_fold_0_hem,path_t_fold_1_hem, path_t_fold_2_hem]
all_plot =  [path_t_fold_0_all, path_t_fold_1_all, path_t_fold_2_all]

image_path_hem_list = readImFolderList(path_hem)
image_path_all_list = readImFolderList(all_plot)

print(image_path_hem_list.shape)
print(image_path_all_list.shape)
y=[]

for x in range (image_path_all_list.shape[0]):
    y.append([1,0])
for x in range (image_path_hem_list.shape[0]):
    y.append([0,1])

# all = [1 0]
# hem = [0 1]

x_in = np.concatenate((image_path_all_list,image_path_hem_list))
y = np.array(y)
print(x_in.shape)
print(y.shape)

(3389,)
(7272,)
(10661,)
(10661, 2)


# Initiate model

In [5]:
model = model()
model.summary()
#model.load_weights('model_weight.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 378, 378, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 189, 189, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 187, 187, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 93, 93, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 91, 91, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 45, 45, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 43, 43, 64)        3

# Training
### Take 50 random images inside the validation data to monitor the training

In [7]:
rando_batch= np.random.randint(low=0,high=x_val.shape[0],size=50)  

accuracy_ =  []
specificty_ = []
sensibility = []
loss_ = []
batch_size= 32

#Manual training

for i in range(100):
    print(i)
    for c in range(20):
        #clear memory
        idx = []
        X= []
        X_t = []
        Y = []
        #take 32 random images in the training database
        idx = np.random.randint(low=0,high=x_in.shape[0],size=batch_size) 
        X = x_in[idx]
        Y = y[idx]
        X_n = norm_and_resize_im(X)
        m_loss = model.train_on_batch(X_n,Y)
        
        if (c==0): #print metrics every 20 iterations
            
            TP = 0 #all = positif
            TN = 0 #hem = negatif
            FP = 0 
            FN = 0
            idx= rando_batch
            X = x_val[idx]
            X_n = norm_and_resize_im(X)
            Y = y_val[idx]
            Y_predict= model.predict(X_n)
            print("validation_process")
            validation_value = 0
            for m in range (0,50):

                y_val_0 = Y[m][0]
                y_val_1 = Y[m][1]
                Y_predict_0 = Y_predict[m][0]
                Y_predict_1 = Y_predict[m][1]

                #Calculate the metrics.
                if (get_max_index(Y[m]) == get_max_index(Y_predict[m])):
                    #trueP or trueN
                    if (Y_predict_0 > Y_predict_1):
                        TP = TP +1
                    else:
                        TN = TN + 1 
                else:
                    if (y_val_0 > y_val_1):
                        FN = FN +1
                    else:
                        FP = FP + 1
            TP = np.round(TP/50,2)
            TN = np.round(TN/50,2)
            FP = np.round(FP/50,2)
            FN = np.round(FN/50,2)

            #Print convolution matrix
            MAT_CONF = [(TP, FP), (FN, TN)]
            print('TP ' ,TP, '  ', 'FP ',FP)
            print('FN ',FN, '  ', 'TN ',TN)
            print('ACCURACY : ',(TP+TN)/(TP+TN+FP+FN))
            accuracy_.append((TP+TN)/(TP+TN+FP+FN)) 
            print('SENSIVITY : ', (TP)/(TP+FN))
            sensibility.append((TP)/(TP+FN))
            print('SPECIFICITY : ', (TN)/(TN+FP))
            specificty_.append((TN)/(TN+FP))
            
            
model.save_weights('name.h5')

    

0
validation_process
TP  0.74    FP  0.26
FN  0.0    TN  0.0
ACCURACY :  0.74
SENSIVITY :  1.0
SPECIFICITY :  0.0
1
validation_process
TP  0.68    FP  0.22
FN  0.06    TN  0.04
ACCURACY :  0.7200000000000001
SENSIVITY :  0.918918918918919
SPECIFICITY :  0.15384615384615385
2
validation_process
TP  0.7    FP  0.22
FN  0.04    TN  0.04
ACCURACY :  0.74
SENSIVITY :  0.9459459459459459
SPECIFICITY :  0.15384615384615385
3
validation_process
TP  0.66    FP  0.22
FN  0.08    TN  0.04
ACCURACY :  0.7000000000000001
SENSIVITY :  0.891891891891892
SPECIFICITY :  0.15384615384615385
4
validation_process
TP  0.74    FP  0.22
FN  0.0    TN  0.04
ACCURACY :  0.78
SENSIVITY :  1.0
SPECIFICITY :  0.15384615384615385
5
validation_process
TP  0.74    FP  0.22
FN  0.0    TN  0.04
ACCURACY :  0.78
SENSIVITY :  1.0
SPECIFICITY :  0.15384615384615385
6
validation_process
TP  0.64    FP  0.22
FN  0.1    TN  0.04
ACCURACY :  0.68
SENSIVITY :  0.8648648648648649
SPECIFICITY :  0.15384615384615385
7
validation

56
validation_process
TP  0.62    FP  0.18
FN  0.12    TN  0.08
ACCURACY :  0.7000000000000001
SENSIVITY :  0.8378378378378378
SPECIFICITY :  0.3076923076923077
57
validation_process
TP  0.66    FP  0.22
FN  0.08    TN  0.04
ACCURACY :  0.7000000000000001
SENSIVITY :  0.891891891891892
SPECIFICITY :  0.15384615384615385
58
validation_process
TP  0.62    FP  0.2
FN  0.12    TN  0.06
ACCURACY :  0.68
SENSIVITY :  0.8378378378378378
SPECIFICITY :  0.23076923076923075
59
validation_process
TP  0.66    FP  0.14
FN  0.08    TN  0.12
ACCURACY :  0.78
SENSIVITY :  0.891891891891892
SPECIFICITY :  0.4615384615384615
60
validation_process
TP  0.7    FP  0.24
FN  0.04    TN  0.02
ACCURACY :  0.72
SENSIVITY :  0.9459459459459459
SPECIFICITY :  0.07692307692307693
61
validation_process
TP  0.7    FP  0.22
FN  0.04    TN  0.04
ACCURACY :  0.74
SENSIVITY :  0.9459459459459459
SPECIFICITY :  0.15384615384615385
62
validation_process
TP  0.62    FP  0.16
FN  0.12    TN  0.1
ACCURACY :  0.72
SENSIVITY :

# Plot graphs 

In [ ]:
plt.figure()
plt.plot(accuracy_)
print(np.sum(accuracy_)/len(accuracy_))
plt.figure()
plt.plot(sensibility)
print(np.sum(sensibility)/len(accuracy_))
plt.figure()
plt.plot(specificty_)
print(np.sum(specificty_)/len(accuracy_))

# Load validation datas

In [5]:
image_path_hem_list_validation = readImFolderList(path_hem_validation)
image_path_all_list_validation = readImFolderList(path_all_validation)

y_val=[]
for x in range (image_path_all_list_validation.shape[0]):
    y_val.append([1,0])

for x in range (image_path_hem_list_validation.shape[0]):
    y_val.append([0,1])
    
x_val = np.concatenate((image_path_all_list_validation,image_path_hem_list_validation))
y_val = np.array(y_val)
print(x_val.shape)
print(y_val.shape)

(1867,)
(1867, 2)


# Predict the output of the validation data

In [ ]:
Y_predict=[]
for m in range(0,x_val.shape[0],2):
    X_n = norm_and_resize_im(x_val[m:m+2])
    Y_val = model.predict(X_n)
    Y_predict.append(Y_val[0])
    Y_predict.append(Y_val[1])

# Save weights

In [ ]:
model.save_weights('final_2.h5')

# Calculate the final convolution matrix and the three metrics.

In [9]:
# all [1 0]
# hem [0 1]
TP = 0 #all = positif
TN = 0 #hem = negatif
FP = 0 
FN = 0
validation_value = 0
final_accuracy_ = []
Y_predict = np.array(Y_predict)
y_val = np.array(y_val)
for m in range (x_val.shape[0]):
    y_val_0 = y_val[m][0]
    y_val_1 = y_val[m][1]
    Y_predict_0 = Y_predict[m][0]
    Y_predict_1 = Y_predict[m][1]
    
    if (get_max_index(y_val[m]) == get_max_index(Y_predict[m])):
        #trueP or trueN
        if (Y_predict_0 > Y_predict_1):
            TP = TP +1
        else:
            TN = TN + 1 
    else:
        if (y_val_0 > y_val_1):
            FN = FN +1
        else:
            FP = FP + 1
TP = np.round(TP/x_val.shape[0],2)
TN = np.round(TN/x_val.shape[0],2)
FP = np.round(FP/x_val.shape[0],2)
FN = np.round(FN/x_val.shape[0],2)


MAT_CONF = [(TP, FP), (FN, TN)]
print('TP ' ,TP, '  ', 'FP ',FP)
print('FN ',FN, '  ', 'TN ',TN)
print('ACCURACY : ',(TP+TN)/(TP+TN+FP+FN))
print('SENSIVITY : ', (TP)/(TP+FN))
print('SPECIFICITY : ', (TN)/(TN+FP))

TP  0.6    FP  0.25
FN  0.05    TN  0.1
ACCURACY :  0.7
SENSIVITY :  0.923076923076923
SPECIFICITY :  0.28571428571428575
